## 移除最后conv最后兩层网络

使用兩個fc, 最好的結果是 72%


In [1]:
import sys
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mping
import torch
import torch.nn as nn
import torchvision.models as models
import torch.utils.data as data
from torchvision import transforms

from data_loader import get_loader
from data_loader import get_encoder_loader
from model import *

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

### set high parameters

In [17]:
batch_size = 256
num_epochs = 200
extract_size = 512
class_size = 61


# 图片格式转化
transform_train = transforms.Compose([
    transforms.Resize([224,224]),
#     transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
transform_vaild = transforms.Compose([
    transforms.Resize([224,224]),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

### load encoded datas

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# exract the images to embedding tensor
# remove the last conv layer
encoder = EncoderCut2Layer()
encoder = encoder.to(device)

# data_loader = get_encoder_loader(transform_train, encoder, device, mode='train', batch_size=batch_size)
# data_loader.dataset.save_to("./bottle_neck/resnet152_train_remove_last_2_conv.h")
data_loader = get_encoder_loader(transform_train, encoder, device, mode='train', batch_size=batch_size, file='./bottle_neck/resnet152_train_remove_last_2_conv.h')

### init model

使用两层网络,结果过拟合,训练准确率很高,验证结果比较差,使用再试

In [7]:
# set the total number of training steps per epoch
total_step = int(len(data_loader.dataset)/batch_size)

classify_model = MultiClassify(extract_size, class_size)
classify_model = classify_model.to(device)

criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# with RMSprop to slow the desent gradient progress
optimizer = torch.optim.Adam(classify_model.parameters(), lr=0.001)

### Load the best trained model,yet!

In [8]:
classify_model.load_state_dict(torch.load('./models/class_multify_rm2layer_last.pkl'))

### time to train model

In [18]:
optimizer = torch.optim.SGD(classify_model.parameters(), lr=0.0001)

In [19]:
classify_model = classify_model.train()
best_acc = 0

for epoch in range(101, num_epochs+101):
    start = time.time()
    for i_step in range(1, total_step+1):
        
        # Ramdomly get samples
        indices = data_loader.dataset.get_train_indices()
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        embeds, targets = next(iter(data_loader))
        
        embeds = embeds.squeeze(1)
        targets = targets.type(torch.LongTensor).to(device)
        
        classify_model.zero_grad()
        
        outputs = classify_model(embeds)
        
        loss = criterion(outputs, targets.view(-1))
        
        loss.backward()
        
        optimizer.step()
        
        if i_step%20 == 0:
            # calculate the status
            predict_result = outputs.argmax(1)
            accuracy = torch.sum(predict_result == targets).item() / batch_size * 100
            best_acc = accuracy if accuracy > best_acc else best_acc
            
            stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Accuracy: %.2f%%, Best_acc: %.2f%%\
            ' % (epoch, num_epochs, i_step, total_step, loss.item(), accuracy, best_acc)
            print('\r' + stats, end='')
            sys.stdout.flush()
    print('\n Epoch {}, spent time:{:.2f}s'.format(epoch, time.time()-start))       
    if epoch%50 == 0:
        torch.save(classify_model.state_dict(), os.path.join('./models', 'class_multify_rm2layer_%d.pkl' % epoch))
torch.save(classify_model.state_dict(), os.path.join('./models', 'class_multify_rm2layer_last.pkl'))

Epoch [101/200], Step [120/127], Loss: 0.6086, Accuracy: 77.73%, Best_acc: 77.73%            
 Epoch 101, spent time:5.68s
Epoch [102/200], Step [120/127], Loss: 0.6904, Accuracy: 73.83%, Best_acc: 81.64%            
 Epoch 102, spent time:5.67s
Epoch [103/200], Step [120/127], Loss: 0.6140, Accuracy: 75.78%, Best_acc: 81.64%            
 Epoch 103, spent time:5.69s
Epoch [104/200], Step [120/127], Loss: 0.5171, Accuracy: 79.69%, Best_acc: 81.64%            
 Epoch 104, spent time:5.88s
Epoch [105/200], Step [120/127], Loss: 0.7540, Accuracy: 73.44%, Best_acc: 81.64%            
 Epoch 105, spent time:5.76s
Epoch [106/200], Step [120/127], Loss: 0.6568, Accuracy: 77.34%, Best_acc: 81.64%            
 Epoch 106, spent time:5.65s
Epoch [107/200], Step [120/127], Loss: 0.6411, Accuracy: 75.39%, Best_acc: 81.64%            
 Epoch 107, spent time:5.70s
Epoch [108/200], Step [120/127], Loss: 0.6657, Accuracy: 76.17%, Best_acc: 81.64%            
 Epoch 108, spent time:5.70s
Epoch [109/200],

Epoch [167/200], Step [120/127], Loss: 0.6528, Accuracy: 77.73%, Best_acc: 83.98%            
 Epoch 167, spent time:5.71s
Epoch [168/200], Step [120/127], Loss: 0.6064, Accuracy: 76.95%, Best_acc: 83.98%            
 Epoch 168, spent time:5.82s
Epoch [169/200], Step [120/127], Loss: 0.6179, Accuracy: 78.12%, Best_acc: 83.98%            
 Epoch 169, spent time:5.68s
Epoch [170/200], Step [120/127], Loss: 0.6298, Accuracy: 76.17%, Best_acc: 83.98%            
 Epoch 170, spent time:5.61s
Epoch [171/200], Step [120/127], Loss: 0.5780, Accuracy: 80.08%, Best_acc: 83.98%            
 Epoch 171, spent time:5.71s
Epoch [172/200], Step [120/127], Loss: 0.5746, Accuracy: 77.73%, Best_acc: 83.98%            
 Epoch 172, spent time:5.60s
Epoch [173/200], Step [120/127], Loss: 0.7401, Accuracy: 72.27%, Best_acc: 83.98%            
 Epoch 173, spent time:5.72s
Epoch [174/200], Step [120/127], Loss: 0.7230, Accuracy: 73.05%, Best_acc: 83.98%            
 Epoch 174, spent time:5.70s
Epoch [175/200],

Epoch [233/200], Step [120/127], Loss: 0.6929, Accuracy: 75.00%, Best_acc: 83.98%            
 Epoch 233, spent time:5.60s
Epoch [234/200], Step [120/127], Loss: 0.6582, Accuracy: 75.39%, Best_acc: 83.98%            
 Epoch 234, spent time:5.68s
Epoch [235/200], Step [120/127], Loss: 0.5994, Accuracy: 78.52%, Best_acc: 83.98%            
 Epoch 235, spent time:5.63s
Epoch [236/200], Step [120/127], Loss: 0.6550, Accuracy: 75.39%, Best_acc: 83.98%            
 Epoch 236, spent time:5.74s
Epoch [237/200], Step [120/127], Loss: 0.6831, Accuracy: 76.95%, Best_acc: 83.98%            
 Epoch 237, spent time:5.74s
Epoch [238/200], Step [120/127], Loss: 0.7351, Accuracy: 74.61%, Best_acc: 83.98%            
 Epoch 238, spent time:5.68s
Epoch [239/200], Step [120/127], Loss: 0.6518, Accuracy: 77.34%, Best_acc: 83.98%            
 Epoch 239, spent time:5.68s
Epoch [240/200], Step [120/127], Loss: 0.7276, Accuracy: 74.22%, Best_acc: 84.38%            
 Epoch 240, spent time:5.67s
Epoch [241/200],

Epoch [299/200], Step [120/127], Loss: 0.6757, Accuracy: 76.17%, Best_acc: 84.38%            
 Epoch 299, spent time:5.67s
Epoch [300/200], Step [120/127], Loss: 0.7087, Accuracy: 73.44%, Best_acc: 84.38%            
 Epoch 300, spent time:5.78s


### How good is the model?

In [11]:
# valid_data_loader = get_encoder_loader(transform_vaild, encoder, device, mode='valid', batch_size=batch_size)
# valid_data_loader.dataset.save_to("./bottle_neck/resnet152_valid_rm1layer.h")
valid_data_loader = get_encoder_loader(transform_vaild, encoder, device, mode='valid', batch_size=batch_size, file="./bottle_neck/resnet152_valid_rm2layer.h")

In [27]:
classify_model.load_state_dict(torch.load('./models/class_multify_rm2layer_300.pkl'))

In [28]:
classify_model = classify_model.eval()
predict = []
for embed, _ in valid_data_loader.dataset:
    p = classify_model(embed).argmax().item()
    predict.append(p)
    
df_refer = valid_data_loader.dataset.refer
df_refer["predict"] = predict
df_refer['correct'] = df_refer.predict == df_refer.disease_class
accuracy = (df_refer.correct == True).sum()/len(df_refer)
print('The final accuracy is %.2f%%.' % (accuracy*100))

The final accuracy is 72.24%.
